In [ ]:
import sys
sys.path.append('..')

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)

import os
os.environ["LANGCHAIN_TRACING_V2"] = "true" 
os.environ["LANGCHAIN_PROJECT"] = "react-meal-planner-demo"

print("✅ Environment setup complete")


In [ ]:
from main_agent import create_meal_planning_agent
from nutrition_optimizer import MealPlan
from food_database import get_food_database
from langchain_core.messages import HumanMessage
import json

# Create the ReAct agent
agent = create_meal_planning_agent()

def create_state():
    return {
        "messages": [],
        "current_meal_plan": MealPlan(),
        "user_profile": {},
        "food_database": get_food_database(),
        "conversation_phase": "gathering_info",
        "optimization_history": []
    }

print("✅ ReAct agent ready")


In [ ]:
def demonstrate_reasoning(user_message, scenario_name):
    """Show how the agent reasons about a user request."""
    print(f"🎯 **Scenario:** {scenario_name}")
    print(f"👤 **User:** {user_message}")
    print("-" * 60)
    
    state = create_state()
    state["messages"].append(HumanMessage(content=user_message))
    
    config = {"configurable": {"thread_id": f"react_demo_{scenario_name.lower().replace(' ', '_')}"}}
    
    # Run the agent
    response = agent.invoke(state, config)
    agent_response = response["messages"][-1]
    
    print(f"🤖 **Agent Response:**")
    print(agent_response.content[:400] + "..." if len(agent_response.content) > 400 else agent_response.content)
    
    # Check if tools were called
    if hasattr(agent_response, 'tool_calls') and agent_response.tool_calls:
        print(f"\n🔧 **Tools Called:** {len(agent_response.tool_calls)}")
        for tool_call in agent_response.tool_calls:
            print(f"- {tool_call['name']}: {tool_call['args']}")
    else:
        print(f"\n💬 **Direct Response** (no tools used)")
    
    print("\n" + "="*80 + "\n")
    return response

# Test different types of requests
scenarios = [
    ("I need help planning meals for the week", "General Request"),
    ("Set my daily calories to 2000 with high protein", "Specific Goal Setting"),
    ("What's a good breakfast under 400 calories?", "Meal Suggestion Request"),
    ("Can you analyze my current meal plan?", "Analysis Request"),
    ("I'm allergic to nuts", "Constraint Declaration")
]

for message, scenario in scenarios:
    demonstrate_reasoning(message, scenario)


In [ ]:
def multi_turn_demo():
    """Demonstrate a complete multi-turn conversation."""
    print("🔄 **Multi-Turn ReAct Conversation Demo**\n")
    
    state = create_state()
    config = {"configurable": {"thread_id": "multi_turn_demo"}}
    
    # Conversation turns
    turns = [
        "I'm a busy professional who wants to eat healthier",
        "I usually have about 20 minutes for meal prep", 
        "I'm trying to lose weight, so around 1600 calories per day",
        "Can you suggest a quick breakfast?",
        "That sounds good! Can you add it to my meal plan?",
        "What about lunch and dinner ideas?"
    ]
    
    for i, user_message in enumerate(turns, 1):
        print(f"**Turn {i}:**")
        print(f"👤 User: {user_message}")
        
        # Add user message to state
        state["messages"].append(HumanMessage(content=user_message))
        
        # Get agent response
        response = agent.invoke(state, config)
        agent_response = response["messages"][-1]
        
        # Update state for next turn
        state = response
        
        # Show reasoning
        print(f"🧠 Agent Reasoning: ", end="")
        if hasattr(agent_response, 'tool_calls') and agent_response.tool_calls:
            tool_names = [tc['name'] for tc in agent_response.tool_calls]
            print(f"Decided to use tools: {', '.join(tool_names)}")
        else:
            print("Decided to respond conversationally")
        
        # Show response (truncated)
        response_preview = agent_response.content[:200] + "..." if len(agent_response.content) > 200 else agent_response.content
        print(f"🤖 Agent: {response_preview}")
        print("-" * 60)
    
    print("\n✅ **Multi-turn conversation complete!**")
    print("Notice how the agent:")
    print("- Builds context across turns")
    print("- Remembers previous information") 
    print("- Adapts tool usage based on conversation flow")
    print("- Maintains natural conversation rhythm")

# Run the demo
multi_turn_demo()


In [ ]:
# Let's examine the tools available to our ReAct agent
from main_agent import (
    generate_meal_suggestions, set_nutrition_goals, 
    optimize_meal_nutrition, analyze_daily_nutrition, save_meal_to_plan
)

tools_info = [
    {
        "name": "generate_meal_suggestions",
        "purpose": "Create meal ideas based on preferences and constraints",
        "when_used": "User asks for meal suggestions, breakfast/lunch/dinner ideas",
        "parameters": ["meal_type", "dietary_preferences", "calorie_target", "avoid_foods"]
    },
    {
        "name": "set_nutrition_goals", 
        "purpose": "Establish daily calorie and macro targets",
        "when_used": "User specifies calorie goals, diet types (high-protein, low-carb)",
        "parameters": ["daily_calories", "diet_type"]
    },
    {
        "name": "optimize_meal_nutrition",
        "purpose": "Adjust meal portions to meet nutritional targets", 
        "when_used": "User wants to optimize existing meals for nutrition",
        "parameters": ["meal_json", "nutrition_goals_json"]
    },
    {
        "name": "analyze_daily_nutrition",
        "purpose": "Review complete daily nutritional intake",
        "when_used": "User asks for nutrition analysis, daily summary",
        "parameters": []
    },
    {
        "name": "save_meal_to_plan",
        "purpose": "Add specific meals to the daily meal plan",
        "when_used": "User wants to save/add meals to their plan",
        "parameters": ["meal_json", "meal_type"]
    }
]

print("🛠️  **ReAct Agent Tool Arsenal**\n")
print(f"{'Tool Name':<25} {'Purpose':<45} {'Key Parameters'}")
print("-" * 100)

for tool in tools_info:
    params = ", ".join(tool["parameters"][:2])  # Show first 2 params
    if len(tool["parameters"]) > 2:
        params += "..."
    print(f"{tool['name']:<25} {tool['purpose']:<45} {params}")

print("\n🧠 **How the Agent Chooses Tools:**")
print("1. **Analyzes user intent** from natural language")
print("2. **Considers conversation context** and previous interactions") 
print("3. **Selects appropriate tool(s)** based on the request type")
print("4. **Executes tools** with extracted parameters")
print("5. **Observes results** and provides natural language response")

print("\n💡 **Key Advantages:**")
print("- No rigid intent classification needed")
print("- Can combine multiple tools in one response")
print("- Adapts tool usage based on conversation flow")
print("- Natural parameter extraction from user messages")


In [ ]:
def compare_approaches():
    """Compare ReAct vs Traditional approaches with examples."""
    
    print("⚖️  **Traditional Intent-Based vs ReAct Comparison**\n")
    
    test_cases = [
        {
            "user_input": "I want a vegetarian breakfast with at least 25g protein under 400 calories",
            "traditional": {
                "step1": "Intent Classification: 'meal_suggestion_request'",
                "step2": "Entity Extraction: meal_type='breakfast', dietary='vegetarian', protein='25g', calories='400'",
                "step3": "Route to predefined meal_suggestion_flow", 
                "step4": "Execute hardcoded breakfast suggestion logic",
                "limitations": ["Rigid flow", "Fixed entity types", "No adaptation"]
            },
            "react": {
                "step1": "LLM analyzes: User wants breakfast suggestion with specific constraints",
                "step2": "Reasoning: Need to call generate_meal_suggestions with extracted parameters", 
                "step3": "Action: Call tool with meal_type='breakfast', dietary_preferences=['vegetarian'], etc.",
                "step4": "Observe results and provide natural response",
                "advantages": ["Flexible reasoning", "Natural parameter extraction", "Contextual adaptation"]
            }
        },
        {
            "user_input": "Actually, make that dairy-free too and show me the nutrition breakdown",
            "traditional": {
                "problem": "Intent unclear - is this modification or new request?",
                "solution": "Would likely fail or require rigid dialog management",
                "limitations": ["Poor context handling", "Cannot adapt mid-conversation", "Requires restart"]
            },
            "react": {
                "reasoning": "LLM understands this modifies previous request",
                "action": "Update constraints and regenerate suggestions + nutrition analysis",
                "advantages": ["Natural context flow", "Seamless adaptation", "No conversation restart needed"]
            }
        }
    ]
    
    for i, case in enumerate(test_cases, 1):
        print(f"**Example {i}:** {case['user_input']}")
        print("\n🔧 **Traditional Approach:**")
        if 'step1' in case['traditional']:
            for step in ['step1', 'step2', 'step3', 'step4']:
                if step in case['traditional']:
                    print(f"  {step}: {case['traditional'][step]}")
            print(f"  ❌ Limitations: {', '.join(case['traditional']['limitations'])}")
        else:
            print(f"  ❌ Problem: {case['traditional']['problem']}")
            print(f"  ⚠️  Solution: {case['traditional']['solution']}")
            print(f"  ❌ Limitations: {', '.join(case['traditional']['limitations'])}")
        
        print("\n🧠 **ReAct Approach:**")
        if 'step1' in case['react']:
            for step in ['step1', 'step2', 'step3', 'step4']:
                if step in case['react']:
                    print(f"  {step}: {case['react'][step]}")
            print(f"  ✅ Advantages: {', '.join(case['react']['advantages'])}")
        else:
            print(f"  🧠 Reasoning: {case['react']['reasoning']}")
            print(f"  🎯 Action: {case['react']['action']}")
            print(f"  ✅ Advantages: {', '.join(case['react']['advantages'])}")
        
        print("\n" + "="*80 + "\n")
    
    print("📊 **Summary Comparison:**")
    print(f"{'Aspect':<20} {'Traditional':<30} {'ReAct':<30}")
    print("-" * 80)
    print(f"{'Flexibility':<20} {'Rigid intent flows':<30} {'Adaptive reasoning':<30}")
    print(f"{'Context Handling':<20} {'Poor mid-conversation':<30} {'Excellent continuity':<30}")
    print(f"{'Parameter Extraction':<20} {'Fixed entity types':<30} {'Natural language parsing':<30}")
    print(f"{'Tool Usage':<20} {'Predefined flows':<30} {'Dynamic selection':<30}")
    print(f"{'User Experience':<20} {'Form-like interaction':<30} {'Natural conversation':<30}")
    print(f"{'Extensibility':<20} {'Add new intents/flows':<30} {'Add new tools easily':<30}")

compare_approaches()


In [ ]:
def test_edge_cases():
    """Test challenging scenarios that would break traditional systems."""
    
    print("🎯 **Edge Case Testing for ReAct Agent**\n")
    
    edge_cases = [
        {
            "scenario": "Ambiguous Request",
            "message": "I need something healthy",
            "challenge": "Very vague - what type of meal? What does 'healthy' mean?"
        },
        {
            "scenario": "Contradictory Constraints", 
            "message": "I want a 3000 calorie meal for weight loss",
            "challenge": "Logical contradiction that needs clarification"
        },
        {
            "scenario": "Complex Multi-Part Request",
            "message": "Plan my meals for tomorrow, I'm vegetarian, allergic to nuts, training for a marathon so need high carbs, prefer Mediterranean food, and have a 1-hour lunch break",
            "challenge": "Multiple constraints and preferences to process simultaneously"
        },
        {
            "scenario": "Context-Dependent Request",
            "message": "Can you make that with more protein instead?", 
            "challenge": "Requires understanding previous conversation context"
        },
        {
            "scenario": "Domain Boundary Testing",
            "message": "What's the weather like today?",
            "challenge": "Outside the meal planning domain"
        }
    ]
    
    for case in edge_cases:
        print(f"🧪 **{case['scenario']}**")
        print(f"👤 Message: \"{case['message']}\"")
        print(f"⚠️  Challenge: {case['challenge']}")
        
        # Test with the agent
        state = create_state()
        state["messages"].append(HumanMessage(content=case['message']))
        config = {"configurable": {"thread_id": f"edge_case_{case['scenario'].lower().replace(' ', '_')}"}}
        
        try:
            response = agent.invoke(state, config)
            agent_response = response["messages"][-1]
            
            print(f"🤖 Agent Response Strategy: ", end="")
            if hasattr(agent_response, 'tool_calls') and agent_response.tool_calls:
                print("Used tools - shows understanding")
            else:
                print("Conversational response - seeks clarification or handles gracefully")
            
            response_preview = agent_response.content[:150] + "..." if len(agent_response.content) > 150 else agent_response.content
            print(f"📝 Response Preview: {response_preview}")
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
        
        print("-" * 70)
    
    print("\n✅ **Edge Case Analysis Summary:**")
    print("ReAct agents handle edge cases well because they:")
    print("1. **Reason about ambiguous requests** and ask for clarification")
    print("2. **Identify contradictions** and guide users toward resolution") 
    print("3. **Break down complex requests** into manageable parts")
    print("4. **Maintain context** for references to previous conversation")
    print("5. **Gracefully handle out-of-domain** requests")
    print("6. **Adapt responses** based on the specific challenge presented")

test_edge_cases()
